In [ ]:

EPOCHS = 10
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter(f"runs/piece_to_move_{timestamp}")

train_data, validation_data = torch.utils.data.random_split(train_data, [1-TEST_PERCENT, TEST_PERCENT])

training_loader = torch.utils.data.dataLoader(train_data, BATCH_SIZE, shuffle=True, pin_memory=True)
validation_loader = torch.utils.data.dataLoader(validation_data, BATCH_SIZE, shuffle=True, pin_memory=True)

def train_one_epoch(epoch_index: int, tb_writer, optimizer, training_loader, loss_fn): 
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(training_loader):
        inputs = data[0]
        labels = data[1]

        optimizer.zero_grad()
        outputs = piece_to_move_net(inputs)

               
        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999: 
            last_loss = running_loss / 1000
            print(f" batch {i + 1}, loss: {last_loss}")
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar("Loss/train", last_loss, tb_x)
            running_loss = 0.

    return last_loss


def train_multiple_epochs(n_epochs, model, writer, validation_loader, loss_fn):
    for epoch in range(EPOCHS): 
        print(f"EPOCH {epoch + 1}: ")

        model.train(True)
        avg_loss = train_one_epoch(epoch, writer)

        model.train(False)
        running_vloss = 0.0
        for i, v_data in enumerate(validation_loader):
            vinputs = v_data[0]
            vlabels = v_data[1]

            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print(f"LOSS train {avg_loss}, valid {avg_vloss}")

        writer.add_scalars("Training vs Validation Loss", {
            "Training": avg_loss, "Validation": avg_vloss}, 
            epoch + 1)
        writer.flush()
        
        if avg_vloss < best_vloss:
            best_vloss = avg_loss
            model_path = f"model_{timestamp}_{epoch}"
            torch.save(model.state_dict(), model_path)

        epoch += 1


In [16]:
import torch
from torch.masked import masked_tensor
import aux_functions
import importlib

importlib.reload(aux_functions)
from aux_functions import *



def generate_mask(tensor)-> list:
    """Generates a mask which contains the position of the pieces that can move"""

    # Initiate a list which will contain the position of the pieces that can move
    positions = []

    # If layer 12 has any 1 it will be whites turn
    if  torch.any(tensor[12] == 1):
        # White pieces are in layers 6 to 11, apply a mask which will be 1 when there is a one
        mask = tensor[0:6] == 1
        print("Whites turn")


    # If layer 13 has any 1 it will be blacks turn
    elif torch.any(tensor[13] == 1):
        # Black pieces are in range 0 to 6
        # Apply a mask, if there is a piece it will be a 1
        mask = tensor[6:12] == 1
        print("Blacks turn")
        
    
    # Sparse tensor to obtain index of the non zero elements
    sparse_coo_mt = mask.to_sparse_coo()

    
    #print(f"Number of movable pieces: {sparse_coo_mt.indices().size(1)}")

    # Access the index of the different elements
    for i in range(sparse_coo_mt.indices().size(1)): # Size 1 is for the number of columns
        # Obtain row and column
        layer, row, col = sparse_coo_mt.indices()[:,i].tolist() # Despite not using layers, we still have to store the value for dimensions problems
        #print(f"Piece at layer {layer}, row {row}, column {col}")  
        
        # Obtain the position in a 64 board list
        position = row*8 + col
        #print(f"Corresponding board position: {position}")
        # Add this position to the final list
        positions.append(position)

    
    return positions

    

import torch


In [32]:
import chess
import torch
import random

# Function to create a sample board
def create_board_with_one_white_piece():
    # Create an empty board
    board = chess.Board(None)  # None initializes an empty board
    
    # Define the piece type (a white pawn in this case)
    piece = chess.Piece(chess.PAWN, chess.WHITE)

    # Randomly select a square from 0 to 63 (representing the board)
    while True:
        square = random.randint(0, 63)
        if board.piece_at(square) is None:  # Check if the square is empty
            board.set_piece_at(square, piece)
            break  # Exit loop once piece is placed

    return board

# Function to convert the chess board to a tensor
def board_to_tensor(board) -> torch.Tensor:
    """Returns a 14x8x8 tensor with ones where each piece is"""
    tensor = torch.zeros((14, 8, 8))

    PIECE_MAP = {chess.PAWN: 0, chess.KNIGHT: 1, chess.BISHOP: 2,
                 chess.ROOK: 3, chess.QUEEN: 4, chess.KING: 5}

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            row, col = divmod(square, 8)
            piece_idx = PIECE_MAP[piece.piece_type]
            color_offset = 0 if piece.color == chess.WHITE else 6
            tensor[piece_idx + color_offset, row, col] = 1

    # Set current turn (layer 12 for white's turn, 13 for black's turn)
    if board.turn == chess.WHITE:
        tensor[12] = 1  # White's turn
    else:
        tensor[13] = 1  # Black's turn

    return tensor

# Create a sample board and convert it to a tensor
board = create_board_with_one_white_piece()
tensor2 = board_to_tensor(board)

# Run the mask generation function
positions = generate_mask(tensor2)

# Output the results
print("Positions of movable pieces:", positions)


Whites turn
Number of movable pieces: 1
Piece at layer 0, row 1, column 1
Corresponding board position: 9
Positions of movable pieces: [9]


In [10]:
data = torch.arange(24).reshape(2, 3, 4)
mask = data % 2 == 0
mt = masked_tensor(data.float(), mask)

masked_tensor.get_da

c:\Users\1\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\masked\maskedtensor\core.py:157: UserWarning: The PyTorch API of MaskedTensors is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our documentation on the torch.masked module for further information about the project.
  warnings.warn(
